## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column
attrition_df.nunique()

,0
Age,43
Attrition,2
BusinessTravel,3
Department,3
DistanceFromHome,29
Education,5
EducationField,6
EnvironmentSatisfaction,4
HourlyRate,71
JobInvolvement,4


In [3]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]

In [6]:
# Create a list of at least 10 column names to use as X data
X_columns = [
    'Age',
    'DistanceFromHome',
    'Education',
    'JobSatisfaction',
    'NumCompaniesWorked',
    'YearsWithCurrManager',
    'PerformanceRating',
    'TotalWorkingYears',
    'WorkLifeBalance',
    'YearsAtCompany'
]

# Create X_df using your selected columns
X_df = attrition_df[X_columns]


# Show the data types for X_df
X_df.dtypes

,0
Age,int64
DistanceFromHome,int64
Education,int64
JobSatisfaction,int64
NumCompaniesWorked,int64
YearsWithCurrManager,int64
PerformanceRating,int64
TotalWorkingYears,int64
WorkLifeBalance,int64
YearsAtCompany,int64


In [7]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=42)


In [ ]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary


,count
OverTime,
No,797
Yes,305


In [8]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [9]:
from sklearn.preprocessing import OneHotEncoder

# Create a OneHotEncoder for the Department column
dept_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Fit the encoder to the training data
dept_encoder.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_dept_encoded = dept_encoder.transform(y_train[['Department']])
y_test_dept_encoded = dept_encoder.transform(y_test[['Department']])

In [11]:
# Create a OneHotEncoder for the Attrition column
attrition_encoder = OneHotEncoder(sparse_output=False, drop='if_binary', handle_unknown='ignore')

# Fit the encoder to the training data
attrition_encoder.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_attrition_encoded = attrition_encoder.transform(y_train[['Attrition']])
y_test_attrition_encoded = attrition_encoder.transform(y_test[['Attrition']])

## Part 2: Create, Compile, and Train the Model

In [10]:
# Find the number of columns in the X training data.
input_features = X_train_scaled.shape[1]

# Create the input layer
inputs = layers.Input(shape=(input_features,))

# Create at least two shared layers
shared_layer1 = layers.Dense(64, activation='relu')(inputs)
shared_layer2 = layers.Dense(32, activation='relu')(shared_layer1)

In [13]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
dept_hidden = layers.Dense(16, activation='relu')(shared_layer2)

# Create the output layer
dept_output = layers.Dense(y_train_dept_encoded.shape[1], activation='softmax', name='dept_output')(dept_hidden)


In [14]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden = layers.Dense(16, activation='relu')(shared_layer2)

# Create the output layer
attrition_output = layers.Dense(1, activation='sigmoid', name='attrition_output')(attrition_hidden)

In [15]:
# Create the model
model = Model(inputs=inputs, outputs=[dept_output, attrition_output])

# Compile the model
model.compile(
    optimizer='adam',
    loss={
        'dept_output': 'categorical_crossentropy',
        'attrition_output': 'binary_crossentropy'
    },
    metrics={
        'dept_output': 'accuracy',
        'attrition_output': 'accuracy'
    }
)

# Summarize the model
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 10)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 64)             │            704 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 32)             │          2,080 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 16)             │            528 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_3 (Dense)           │ (None, 16)             │            528 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dept_output (Dense)       │ (None, 3)              │             51 │ dense_2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_output (Dense)  │ (None, 1)              │             17 │ dense_3[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 3,908 (15.27 KB)

 Trainable params: 3,908 (15.27 KB)

 Non-trainable params: 0 (0.00 B)

In [16]:
# Train the model
history = model.fit(
    X_train_scaled,
    {'dept_output': y_train_dept_encoded, 'attrition_output': y_train_attrition_encoded},
    epochs=100,
    batch_size=32,
    validation_data=(
        X_test_scaled,
        {'dept_output': y_test_dept_encoded, 'attrition_output': y_test_attrition_encoded}
    ),
    verbose=1
)

Epoch 1/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - attrition_output_accuracy: 0.5528 - attrition_output_loss: 0.6659 - dept_output_accuracy: 0.5284 - dept_output_loss: 1.0016 - loss: 1.6674 - val_attrition_output_accuracy: 0.8696 - val_attrition_output_loss: 0.4247 - val_dept_output_accuracy: 0.6522 - val_dept_output_loss: 0.8402 - val_loss: 1.2496
Epoch 2/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - attrition_output_accuracy: 0.8280 - attrition_output_loss: 0.4682 - dept_output_accuracy: 0.6496 - dept_output_loss: 0.8032 - loss: 1.2712 - val_attrition_output_accuracy: 0.8696 - val_attrition_output_loss: 0.3945 - val_dept_output_accuracy: 0.6495 - val_dept_output_loss: 0.8239 - val_loss: 1.2037
Epoch 3/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - attrition_output_accuracy: 0.8275 - attrition_output_loss: 0.4447 - dept_output_accuracy: 0.6277 - dept_output_loss: 0.7911 - loss: 1.2355 - val_attrition_output_accuracy: 0.8696 - val_attrition_output_loss: 0.3872 - val_dept_output_accurac

In [17]:
# Evaluate the model with the testing data
test_results = model.evaluate(
    X_test_scaled,
    {'dept_output': y_test_dept_encoded, 'attrition_output': y_test_attrition_encoded},
    verbose=1
)

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - attrition_output_accuracy: 0.8007 - attrition_output_loss: 0.5647 - dept_output_accuracy: 0.4863 - dept_output_loss: 1.2833 - loss: 1.8440 


In [18]:
# Print the accuracy for both department and attrition
print(f"Department Accuracy: {test_results[3]}")
print(f"Attrition Accuracy: {test_results[4]}")

Department Accuracy: 0.8097826242446899
Attrition Accuracy: 0.49728259444236755


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. Not for attrition, because the attrition and department data are imbalanced, as seen in the code block below.
2. We used softmax for Department because we were working with multi-class classification with mutually exclusive classes. We used sigmoid for attrition because we were working with a binary classification, either leaving or staying.
3. We could improve the model by addressing the class imbalance problem. We could adjust the model architecture, such as adding layers or neurons. We could do some hyperparameter tuning or feature engineering.

In [19]:
print(y_df['Attrition'].value_counts(normalize=True))

Attrition
No     0.838776
Yes    0.161224
Name: proportion, dtype: float64


In [20]:
print(y_df['Department'].value_counts(normalize=True))

Department
Research & Development    0.653741
Sales                     0.303401
Human Resources           0.042857
Name: proportion, dtype: float64
